In [1]:
using LinearAlgebra

m = 100;

A = round.(rand(m,m)*10);

In [2]:
function par_part(v, w)
    # Map v to w then divide by magnitude of w
    pp = (v'w / w'w) .* w
    return pp
end

function gram_schmidt(matrix)

    for i = 1:m
        # Current column as vector
        v = copy(matrix[:,i])

        if i > 1
            for k in i-1:-1:1
                # Previous column as vector
                w = matrix[:,k]
                # Get parallel part of current column, with respect to previous column
                v_wp = par_part(v, w)
                # Subtract parallel part of previous column from current column to get orthogonal part
                v -= v_wp
            end    
        end
        # Normalize orthogonal vector and add to Q matrix
        matrix[:,i] = v / norm(v)
    end

    return matrix
end



gram_schmidt (generic function with 1 method)

In [3]:
Q = gram_schmidt(A)

100×100 Matrix{Float64}:
 0.177836   -0.0394645   -0.132241     0.0608943  …   0.0871363    0.241888
 0.0177836   0.186787    -0.00511099  -0.052146      -0.017863     0.179803
 0.0711343  -0.0366881   -0.0224704    0.121039      -0.0406797   -0.0784968
 0.142269   -0.125632     0.116848    -0.0453511      0.152948     0.168344
 0.106701   -0.0027764    0.090254    -0.111519       0.0327912    0.0239178
 0.0355671   0.00778384   0.179332     0.0926199  …   2.59503e-5  -0.0775352
 0.106701   -0.0550322    0.109169     0.0919464      0.055535     0.0762944
 0.0711343   0.0416956    0.177756    -0.12381        0.0902887   -0.119884
 0.0355671   0.164551     0.0940122   -0.0430088     -0.143451    -0.0929855
 0.106701    0.0494794   -0.0715352    0.114175       0.149197    -0.2093
 ⋮                                                ⋱               
 0.0         0.0261279    0.219142     0.143823      -0.137424     0.176327
 0.106701   -0.133416     0.0518164    0.15458       -0.0331945    0.

In [4]:
# Check if Q'Q = I
Int.(round.(Q'Q))

100×100 Matrix{Int64}:
 1  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮        ⋱        ⋮              ⋮           
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0    

In [5]:
# Test against built-in QR decomposition
F = qr(A);

abs.(round.(Q, digits=2)) == abs.(round.(F.Q, digits=2))

true

## Inverse Via QR decomposition

In [6]:
# Get R
R = Q'A;

In [7]:
# Testing A = QR for fun
round.(A, digits=3) == round.(Q*R, digits=3)

true

In [8]:
A_inv = R^-1*Q';

In [9]:
# A^-1 
round.(inv(A), digits=3) == round.(A_inv, digits=3)

true

## Sherman-Morrison formula

In [10]:
m = 5
a = rand(m);
b = rand(m);

In [11]:
A = I(m) - a*b'

5×5 Matrix{Float64}:
  0.982953    -0.0263908   -0.11544     -0.119699   -0.106811
 -0.0364695    0.943541    -0.246965    -0.256076   -0.228505
 -0.00321223  -0.00497287   0.978247    -0.0225551  -0.0201267
 -0.00141249  -0.00218668  -0.00956509   0.990082   -0.00885013
 -0.03604     -0.0557938   -0.244056    -0.25306     0.774186

In [12]:
A_inv = I(m) + (a*b')/(1-a'b)

5×5 Matrix{Float64}:
 1.02548     0.0394475   0.172554   0.178919   0.159656
 0.0545127   1.08439     0.36915    0.382768   0.341557
 0.00480147  0.00743318  1.03251    0.0337142  0.0300843
 0.00211131  0.00326853  0.0142974  1.01482    0.0132287
 0.0538707   0.0833975   0.364802   0.378261   1.33753

In [13]:
round.(abs.(A*A_inv))

5×5 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

## A'A == R'R

In [14]:
A = rand(m, m)

5×5 Matrix{Float64}:
 0.904671  0.574239  0.922944  0.276952  0.935443
 0.320396  0.948513  0.796817  0.322742  0.248688
 0.173036  0.720581  0.422534  0.234014  0.0695985
 0.445189  0.270391  0.296728  0.603514  0.478212
 0.592059  0.664268  0.971452  0.552025  0.412144

In [15]:
decomp = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor:
5×5 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.738722   0.33585    -0.090793    0.577201   -0.00950471
 -0.261624  -0.694883    0.0856455   0.0720866  -0.660423
 -0.141295  -0.602806   -0.431046    0.11276     0.646642
 -0.363526   0.178588   -0.595618   -0.666048   -0.19384
 -0.483454  -0.0952524   0.666228   -0.453109    0.328681
R factor:
5×5 Matrix{Float64}:
 -1.22464  -1.19361   -1.52749   -0.808364   -1.13902
  0.0      -0.915603  -0.53797   -0.217119    0.145551
  0.0       0.0        0.272788  -0.0900636  -0.103882
  0.0       0.0        0.0       -0.442587    0.0604555
  0.0       0.0        0.0        0.0        -0.0853577

In [16]:
decomp.R'decomp.R

5×5 Matrix{Float64}:
 1.49975   1.46174  1.87063  0.989957  1.3949
 1.46174   2.26303  2.31579  1.16366   1.22628
 1.87063   2.31579  2.69704  1.327     1.63321
 0.989957  1.16366  1.327    0.904588  0.871743
 1.3949    1.22628  1.63321  0.871743  1.34029

In [17]:
A'A

5×5 Matrix{Float64}:
 1.49975   1.46174  1.87063  0.989957  1.3949
 1.46174   2.26303  2.31579  1.16366   1.22628
 1.87063   2.31579  2.69704  1.327     1.63321
 0.989957  1.16366  1.327    0.904588  0.871743
 1.3949    1.22628  1.63321  0.871743  1.34029

In [18]:
round.(decomp.R'decomp.R, digits = 4) == round.(A'A, digits = 4)

true

In [19]:
decomp.Q'decomp.Q

5×5 Matrix{Float64}:
  1.0          5.55112e-17  6.93889e-17   0.0          4.85723e-17
  0.0          1.0          4.16334e-17  -5.55112e-17  0.0
 -2.77556e-17  0.0          1.0           1.52656e-16  3.33067e-16
  1.11022e-16  0.0          1.11022e-16   1.0          0.0
  5.55112e-17  1.11022e-16  1.11022e-16  -1.11022e-16  1.0

## Least Squares via RREF

In [23]:
m = 10
n = 3

3

In [24]:
X = randn(m,n)
y = randn(m,1) 

10×1 Matrix{Float64}:
 -0.2199921395118592
  0.7625898981359973
  0.6067605671204026
 -0.23240642353312665
  0.30881582106306105
 -0.27144357046779083
 -0.0797362604196493
  0.36372571938262854
 -1.8635071213787948
  2.2030366320146615

In [26]:
using RowEchelon

rref([X y])

10×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [35]:
# Reapply using the normal equation
betas1 = rref([X'X X'y])

3×4 Matrix{Float64}:
 1.0  0.0  0.0  -0.0106386
 0.0  1.0  0.0   0.512908
 0.0  0.0  1.0   0.157001

In [36]:
# Compare to left-inverse
betas2 = (X'X)\(X'y)

3×1 Matrix{Float64}:
 -0.01063857645579934
  0.5129079015258228
  0.15700055826260223

In [37]:
betas3 = X\y

3×1 Matrix{Float64}:
 -0.0106385764557996
  0.5129079015258227
  0.15700055826260226